# Modeling extreme chances

The originating question here, if you're playing a game where you're offered a 2% chance, what is the probability of losing 600 times in a row?

Normally when computing probabilities we'd look at this as the probability of not winning at least once in 600 tries.
$$P(\text{winning}) = p = 0.02$$
$$P(\text{not winning}) = P(\text{losing})= {\neg}p = 0.98$$

Independent probabilities multiply together so the chances of not winning twice in a row would be as follows:

$$P(\text{losing twice}) = {\neg}p^2 = 0.9604$$

The probability of losing $n$ times in a row would be given by the following:

$$P(\text{losing $n$ times}) = {\neg}p^n$$

The probability  of losing 600 times in a row would therefore be calculated like this:

$$P(\text{losing 600 times}) = {\neg}p^600 = 0.98^600 = 5.4405826910255264834083104415561140853482895765220476138082×10^{(-6)}$$

Such tiny numbers can be difficult to understand intuitively so we should see about discovering them through simulation.
However, such a small chance could take a long time to occur naturally in a simulation so let's test a simpler case first

## Simulating the simple case

import random
from collections import Counter

First let's model this as if it were a 100 sided die.  This is a discrete distribution and may not be equivilent to a continuous distribution that most computer programs use, so we might want to check them for equivilency later.

In [1]:
import random
from time import perf_counter_ns
from collections import Counter

In [2]:
d_100_rolls = Counter((random.randint(1, 100) for _ in range(1_000_000)))
print(sorted(d_100_rolls.most_common()))

[(1, 10081), (2, 10033), (3, 9932), (4, 10007), (5, 10030), (6, 10127), (7, 10068), (8, 10114), (9, 9979), (10, 10081), (11, 10142), (12, 9974), (13, 9833), (14, 9911), (15, 10083), (16, 10000), (17, 9985), (18, 9897), (19, 10001), (20, 10161), (21, 9936), (22, 10122), (23, 10079), (24, 10201), (25, 9956), (26, 10064), (27, 9935), (28, 10082), (29, 10012), (30, 9883), (31, 10156), (32, 9927), (33, 10126), (34, 9914), (35, 9898), (36, 10134), (37, 10044), (38, 9915), (39, 10078), (40, 9979), (41, 10025), (42, 10042), (43, 10152), (44, 10218), (45, 10038), (46, 9989), (47, 9935), (48, 10069), (49, 9865), (50, 9960), (51, 10009), (52, 9848), (53, 10070), (54, 9915), (55, 10160), (56, 9854), (57, 10036), (58, 9904), (59, 9996), (60, 10040), (61, 10047), (62, 10010), (63, 9876), (64, 10080), (65, 9905), (66, 10061), (67, 9968), (68, 9991), (69, 9885), (70, 10072), (71, 10011), (72, 10011), (73, 9981), (74, 9992), (75, 10060), (76, 9984), (77, 10027), (78, 10100), (79, 9769), (80, 9945), (81

The keys of the Counter object represent the results of the rolls, and they are the intigers 1-100.  The values are the number of times each result was rolled and they're not exactly identical but they are very similar.  This is what we'd epxect from a simulation of a 10-sided die.  So let's see how often we can roll greater than 2 - our 2% chance of winning - twice in a row out of say, 1M attempts.

In [3]:
losses = 0
total_tries = 1_000_000
win_value = 2
num_rolls = 2

for _ in range(total_tries):
    for _ in range(num_rolls):
        current_roll = random.randint(1,100)
        if current_roll <= win_value:
            break
    else:
        losses += 1
print(f"{losses=} Probability of losing = {(p_exp:=losses/total_tries)} difference between experimental and calculated results: {p_exp-0.98**2}")

losses=960680 Probability of losing = 0.96068 difference between experimental and calculated results: 0.000280000000000058


When I run this simulation on my machine, the difference between the experimental and calculated results is about 0.0002, which is very small.

In [4]:
del losses
del total_tries
del win_value
del num_rolls

## Simulating the extreme case

Fist, let's see how many tries it takes to loose six hundred times in a row even once.

In [5]:
win_value = 2
sides = 100
total_tries = 0
num_losses = 600
finished = False

start_time = perf_counter_ns()
while not finished:
    for _ in range(num_losses):
        current_roll = random.randint(1, sides)
        if current_roll <= win_value:
            # This try ended before reaching 600 losses in a row
            total_tries += 1
            break
    else:
        # We can only get here if the for loop did not encounter a break
        total_tries += 1 # The run where we finally lose 600 times in a row still counts.
        break # this breaks out of the while True loop
end_time = perf_counter_ns()

print(f"it took {total_tries} tries and {end_time-start_time}ns to lose {num_losses} times in a row with a target number of {win_value} or lower on a d{sides}")

it took 151220 tries and 1577319300ns to lose 600 times in a row with a target number of 2 or lower on a d100


In [6]:
del win_value
del sides
del total_tries
del num_losses
del finished

It didn't actually take as long as I expected to reach 600 failures in a row, less than one second,so more elaborate simulations are reasonable.

Let's do a probability mass function.

## Experimental probability mass function

A probability mass function or PMF is the probability that some discrete random variable will take on a specific value.  We can build one experimentally by first making a frequency table of many trials, and then normalizing it.

In [7]:
win_value = 2
sides = 100
num_trials = 10_000_000

freq_counter = Counter()

freq_time_start = perf_counter_ns()
for trial in range(num_trials):
    current_trial_rolls = 1
    while((current_roll:= random.randint(1, sides)) > win_value):
        current_trial_rolls += 1
    freq_counter[current_trial_rolls] += 1
freq_time_end = perf_counter_ns()

print(f"took {freq_time_end - freq_time_start}ns to run {num_trials} trials")

took 109369411400ns to run 10000000 trials


In [8]:
sorted_freq_items = sorted(freq_counter.items())

In [9]:
print(sorted_freq_items[-40:])

[(601, 3), (604, 1), (605, 3), (606, 4), (607, 3), (608, 2), (609, 1), (610, 1), (611, 1), (613, 2), (614, 1), (616, 2), (617, 3), (618, 1), (620, 2), (621, 2), (624, 3), (627, 2), (631, 2), (632, 2), (636, 1), (637, 1), (638, 1), (639, 1), (641, 2), (644, 1), (652, 1), (664, 1), (673, 1), (674, 1), (682, 1), (684, 1), (707, 1), (713, 1), (726, 1), (728, 1), (731, 1), (732, 1), (746, 1), (843, 1)]


In [10]:
big_fail_freqs = [(result, freq) for (result, freq) in sorted_freq_items if result > 600]
num_big_fail_trials = sum((freq for (result, freq) in big_fail_freqs))

In [13]:
print(f"The calculated probability of failing 600 times is {(calculated_prob:=pow(0.98, 600))} so out of {num_trials:,} trials we would expect {(expected_big_fails:=calculated_prob * num_trials)} such events."
      f"In simulation we observed {num_big_fail_trials} for a difference of {num_big_fail_trials - expected_big_fails}")

The calculated probability of failing 600 times is 5.440582691025467e-06 so out of 10,000,000 trials we would expect 54.40582691025467 such events.In simulation we observed 62 for a difference of 7.594173089745333
